In [1]:
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("Using only CPU")
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

logging.getLogger('tensorflow').setLevel(logging.FATAL)

from dataset.dataset_loader import NerProcessor, FewNERDProcessor, create_tf_dataset_for_client, split_to_tf_datasets, batch_features
from utils.fl_utils import *

import tensorflow as tf
from tqdm.notebook import tqdm

import numpy as np

from models.model import build_BertNer, MaskedSparseCategoricalCrossentropy
from tokenization import FullTokenizer



# Pretrained models
TINY = 'uncased_L-2_H-128_A-2'
TINY_1 = 'uncased_L-4_H-128_A-2'
TINY_12 = 'uncased_L-12_H-128_A-2'
MINI = 'uncased_L-4_H-256_A-4'
SMALL = 'uncased_L-4_H-512_A-8'
MEDIUM = 'uncased_L-8_H-512_A-8'
BASE = 'uncased_L-12_H-768_A-12'

TINY_8_128='uncased_L-8_H-128_A-2'

MODEL = os.path.join("models", TINY_12)
SEQ_LEN = 128
BATCH_SIZE = 32
PRETRAINED = False

processor = NerProcessor('dataset/conll')
# processor = FewNERDProcessor('dataset/few_nerd')
tokenizer = FullTokenizer(os.path.join(MODEL, "vocab.txt"), True)
train_features = processor.get_train_as_features(SEQ_LEN, tokenizer)
eval_features = processor.get_test_as_features(SEQ_LEN, tokenizer)


def eval_model(model, eval_data, do_print=True):
    return evaluate(model, eval_data, 
                    processor.get_label_map(), 
                    processor.token_ind('O'), 
                    processor.token_ind('[SEP]'),
                    processor.token_ind('[PAD]'), 
                    do_print=do_print)
    
eval_data_batched = batch_features(eval_features, processor.get_labels(), SEQ_LEN, tokenizer, batch_size=64)


Using only CPU


In [2]:
m = build_BertNer(MODEL, processor.label_len(), SEQ_LEN)

In [3]:
bm = m.layers[3]
bm.embedding_postprocessor.output_layer_norm.trainable = False
for l in bm.encoder.layers:
    l.attention_layer_norm.trainable = False
    l.output_layer_norm.trainable = False

In [4]:
for i in range(len(m.layers[3].weights)):
    print(m.layers[3].weights[i].name)

bert_model/word_embeddings/embeddings:0
bert_model/embedding_postprocessor/type_embeddings:0
bert_model/embedding_postprocessor/position_embeddings:0
bert_model/encoder/layer_0/self_attention/query/kernel:0
bert_model/encoder/layer_0/self_attention/query/bias:0
bert_model/encoder/layer_0/self_attention/key/kernel:0
bert_model/encoder/layer_0/self_attention/key/bias:0
bert_model/encoder/layer_0/self_attention/value/kernel:0
bert_model/encoder/layer_0/self_attention/value/bias:0
bert_model/encoder/layer_0/self_attention_output/kernel:0
bert_model/encoder/layer_0/self_attention_output/bias:0
bert_model/encoder/layer_0/intermediate/kernel:0
bert_model/encoder/layer_0/intermediate/bias:0
bert_model/encoder/layer_0/output/kernel:0
bert_model/encoder/layer_0/output/bias:0
bert_model/encoder/layer_1/self_attention/query/kernel:0
bert_model/encoder/layer_1/self_attention/query/bias:0
bert_model/encoder/layer_1/self_attention/key/kernel:0
bert_model/encoder/layer_1/self_attention/key/bias:0
bert

# Train single

In [5]:
@tf.function
def train_batch(model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss = model.loss(y, logits)
    grad = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(grad, model.trainable_variables))

def train_single(epochs=1, lr=5e-5, batch_size=32, pretrained=True):
    model = build_BertNer(MODEL, processor.label_len(), SEQ_LEN)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss=MaskedSparseCategoricalCrossentropy()) # reduction=tf.keras.losses.Reduction.NONE))
    """
    bm = model.layers[3]
    bm.embedding_postprocessor.output_layer_norm.trainable = False
    for l in bm.encoder.layers:
        l.attention_layer_norm.trainable = False
        l.output_layer_norm.trainable = False
    """
    
    if pretrained:
        restore_model_ckpt(model, MODEL)
    
    data = split_to_tf_datasets(train_features, 1, batch_size)[0]
    for e in range(epochs):
        for x, y in tqdm(data, position=0, leave=False, desc="Training"):
            train_batch(model, x, y)
        eval_model(model, eval_data_batched)
    return model

In [ ]:
train_single(50, pretrained=False) # tiny 12

Training:   0%|          | 0/438 [00:00<?, ?it/s]

In [3]:
train_single(50, pretrained=False) # tiny 12 no norm

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.2867    0.6025    0.3885      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.2045    0.0054    0.0106      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.2856    0.1795    0.2205      5648
   macro avg     0.1228    0.1520    0.0998      5648
weighted avg     0.1448    0.1795    0.1178      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

...

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.6279    0.7182    0.6700      1668
        MISC     0.5880    0.5570    0.5721       702
         ORG     0.4623    0.4756    0.4688      1661
         PER     0.3438    0.4261    0.3806      1617

   micro avg     0.4881    0.5432    0.5142      5648
   macro avg     0.5055    0.5442    0.5229      5648
weighted avg     0.4929    0.5432    0.5158      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
train_single(50, pretrained=False) tiny

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.0000    0.0000    0.0000      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.0000    0.0000    0.0000      5648
   macro avg     0.0000    0.0000    0.0000      5648
weighted avg     0.0000    0.0000    0.0000      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

...

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.7006    0.7128    0.7067      1668
        MISC     0.5108    0.5741    0.5406       702
         ORG     0.3920    0.4612    0.4238      1661
         PER     0.3245    0.3704    0.3459      1617

   micro avg     0.4704    0.5235    0.4956      5648
   macro avg     0.4820    0.5296    0.5042      5648
weighted avg     0.4786    0.5235    0.4996      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
train_single(10, pretrained=False)

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.3440    0.5162    0.4129      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.3672    0.0674    0.1139      1661
         PER     0.0750    0.0019    0.0036      1617

   micro avg     0.3427    0.1728    0.2298      5648
   macro avg     0.1966    0.1464    0.1326      5648
weighted avg     0.2311    0.1728    0.1565      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

...

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.6404    0.7698    0.6992      1668
        MISC     0.5787    0.5869    0.5827       702
         ORG     0.3959    0.4040    0.3999      1661
         PER     0.3609    0.4700    0.4083      1617

   micro avg     0.4797    0.5536    0.5141      5648
   macro avg     0.4939    0.5577    0.5225      5648
weighted avg     0.4808    0.5536    0.5134      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
train_single(10)

Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.3871    0.7500    0.5106      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.2898    0.0494    0.0844      1661
         PER     0.1206    0.2468    0.1620      1617

   micro avg     0.2538    0.3067    0.2778      5648
   macro avg     0.1994    0.2615    0.1893      5648
weighted avg     0.2341    0.3067    0.2220      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.8195    0.8711    0.8445      1668
        MISC     0.6534    0.6795    0.6662       702
         ORG     0.6863    0.6954    0.6908      1661
         PER     0.8432    0.8479    0.8455      1617

   micro avg     0.7667    0.7890    0.7777      5648
   macro avg     0.7506    0.7735    0.7618      5648
weighted avg     0.7665    0.7890    0.7774      5648



Training:   0%|          | 0/438 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.8201    0.8747    0.8465      1668
        MISC     0.6499    0.6795    0.6643       702
         ORG     0.6908    0.6899    0.6904      1661
         PER     0.8256    0.8609    0.8429      1617

   micro avg     0.7637    0.7921    0.7777      5648
   macro avg     0.7466    0.7762    0.7610      5648
weighted avg     0.7625    0.7921    0.7769      5648

